# Precision Oncology: Genomic Drug Sensitivity Prediction

## Preprocessing

In [15]:
#import libraries
import pandas as pd
import numpy as np
import janitor
import os

import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator

In [16]:
#read CSV containing drug data
drug_data = pd.read_csv("data/drug_data_raw.csv")
drug_data

,Drug Name,Drug ID,Cell Line Name,Cosmic ID,TCGA Classification,Tissue,Tissue Sub-type,IC50,AUC,Max Conc,RMSE,Z score,Dataset Version
0,Camptothecin,1003,PFSK-1,683667,MB,nervous_system,medulloblastoma,-1.463887,0.930220,0.1,0.089052,0.433123,GDSC2
1,Camptothecin,1003,A673,684052,UNCLASSIFIED,soft_tissue,rhabdomyosarcoma,-4.869455,0.614970,0.1,0.111351,-1.421100,GDSC2
2,Camptothecin,1003,ES5,684057,UNCLASSIFIED,bone,ewings_sarcoma,-3.360586,0.791072,0.1,0.142855,-0.599569,GDSC2
3,Camptothecin,1003,ES7,684059,UNCLASSIFIED,bone,ewings_sarcoma,-5.044940,0.592660,0.1,0.135539,-1.516647,GDSC2
4,Camptothecin,1003,EW-11,684062,UNCLASSIFIED,bone,ewings_sarcoma,-3.741991,0.734047,0.1,0.128059,-0.807232,GDSC2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
243461,N-acetyl cysteine,2499,SNU-175,1659928,COREAD,digestive_system,large_intestine,10.127082,0.976746,2000.0,0.074498,0.156872,GDSC2
243462,N-acetyl cysteine,2499,SNU-407,1660034,COREAD,digestive_system,large_intestine,8.576377,0.913378,2000.0,0.057821,-1.626959,GDSC2
243463,N-acetyl cysteine,2499,SNU-61,1660035,COREAD,digestive_system,large_intestine,10.519636,0.975001,2000.0,0.058090,0.608442,GDSC2
243464,N-acetyl cysteine,2499,SNU-C5,1674021,COREAD,digestive_system,large_intestine,10.694579,0.969969,2000.0,0.101013,0.809684,GDSC2


In [17]:
# read CSV containing genetic data
genetic_data = pd.read_csv("data/genetic_data_raw.csv",
                          dtype={
                              'Recurrent Gain Loss': str,
                              'Genes in Segment': str
                          })
genetic_data

,Cell Line Name,COSMIC ID,GDSC Desc1,GDSC Desc2,TCGA Desc,Genetic Feature,IS Mutated,Recurrent Gain Loss,Genes in Segment
0,CAL-29,1290730,urogenital_system,bladder,BLCA,CDC27_mut,0,NaN,NaN
1,CAL-29,1290730,urogenital_system,bladder,BLCA,CDC73_mut,0,NaN,NaN
2,CAL-29,1290730,urogenital_system,bladder,BLCA,CDH1_mut,0,NaN,NaN
3,CAL-29,1290730,urogenital_system,bladder,BLCA,CDK12_mut,0,NaN,NaN
4,CAL-29,1290730,urogenital_system,bladder,BLCA,CDKN1A_mut,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
697995,UWB1.289,1480374,urogenital_system,ovary,OV,HLA.B_mut,0,NaN,NaN
697996,UWB1.289,1480374,urogenital_system,ovary,OV,HNF1A_mut,0,NaN,NaN
697997,UWB1.289,1480374,urogenital_system,ovary,OV,HRAS_mut,0,NaN,NaN
697998,UWB1.289,1480374,urogenital_system,ovary,OV,HSPA8_mut,0,NaN,NaN


In [18]:
# clean nmmes using janitor
drug_data = drug_data.clean_names()
genetic_data = genetic_data.clean_names()

## Initial Data Exploration

In [19]:
# print shape of drug_data
drug_data.shape

(243466, 13)

In [20]:
# identify types within drug_data
drug_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243466 entries, 0 to 243465
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   drug_name            243466 non-null  object 
 1   drug_id              243466 non-null  int64  
 2   cell_line_name       243466 non-null  object 
 3   cosmic_id            243466 non-null  int64  
 4   tcga_classification  242399 non-null  object 
 5   tissue               243107 non-null  object 
 6   tissue_sub_type      243107 non-null  object 
 7   ic50                 243466 non-null  float64
 8   auc                  243466 non-null  float64
 9   max_conc             243466 non-null  float64
 10  rmse                 243466 non-null  float64
 11  z_score              243466 non-null  float64
 12  dataset_version      243466 non-null  object 
dtypes: float64(5), int64(2), object(6)
memory usage: 24.1+ MB


In [21]:
# basic statistics for drug_data
drug_data.describe()

,drug_id,cosmic_id,ic50,auc,max_conc,rmse,z_score
count,243466.000000,2.434660e+05,243466.000000,243466.000000,243466.000000,243466.000000,2.434660e+05
mean,1595.325466,9.920673e+05,2.814039,0.881876,23.421608,0.082867,6.581099e-18
std,397.918457,2.209516e+05,2.774684,0.148429,158.160555,0.042821,9.993919e-01
min,1003.000000,6.836670e+05,-8.747724,0.006282,0.010000,0.003274,-8.254501e+00
25%,1168.000000,9.068050e+05,1.497959,0.848868,3.000000,0.051090,-6.572931e-01
50%,1632.000000,9.097200e+05,3.237408,0.944192,10.000000,0.076114,1.026455e-02
75%,1912.000000,1.240144e+06,4.707872,0.974946,10.000000,0.106209,6.560839e-01
max,2499.000000,1.789883e+06,13.820189,0.998904,2000.000000,0.299984,7.978776e+00


In [22]:
# print shape of genetic_data
genetic_data.shape

(698000, 9)

In [23]:
# identify types within genetic_data
genetic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698000 entries, 0 to 697999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   cell_line_name       698000 non-null  object
 1   cosmic_id            698000 non-null  int64 
 2   gdsc_desc1           696530 non-null  object
 3   gdsc_desc2           696530 non-null  object
 4   tcga_desc            694210 non-null  object
 5   genetic_feature      698000 non-null  object
 6   is_mutated           698000 non-null  int64 
 7   recurrent_gain_loss  409700 non-null  object
 8   genes_in_segment     409700 non-null  object
dtypes: int64(2), object(7)
memory usage: 47.9+ MB


In [24]:
# basic statistics for genetic_data
genetic_data.describe()

,cosmic_id,is_mutated
count,6.980000e+05,698000.000000
mean,9.951772e+05,0.040358
std,2.242358e+05,0.196798
min,6.836670e+05,0.000000
25%,9.068070e+05,0.000000
50%,9.097270e+05,0.000000
75%,1.240151e+06,0.000000
max,1.789883e+06,1.000000


## SMILES

In [25]:
# obtain SMILES for selected drugs

smiles_path = "data/cached_drug_smiles.csv"

if os.path.exists(smiles_path):
    print("Loading SMILES from file cache...")
    drug_data = pd.read_csv(smiles_path)
else:
    print("SMILES cache not found. Fetching from PubChem API...")
    
    unique_drugs = drug_data['drug_name'].unique()
    
    drug_smiles = {}
    for drug in unique_drugs:
        try:
            compound = pcp.get_compounds(drug, 'name')[0]
            drug_smiles[drug] = compound.smiles
        except (IndexError, pcp.PubChemHTTPError):
            print(f"Could not find SMILES for: {drug}")
            drug_smiles[drug] = None
    
    drug_data['smiles'] = drug_data['drug_name'].map(drug_smiles)

    drug_data.to_csv(smiles_path, index=False)
    
    drug_data = drug_data.dropna(subset=['smiles'])
    print(f"Drugs with SMILES: {drug_data['drug_name'].nunique()}")

Loading SMILES from file cache...


In [26]:
# generate fingerprints
mfgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=1024)

def get_fingerprint(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return np.zeros(1024, dtype=int)
        return mfgen.GetFingerprintAsNumPy(mol)
    except Exception as e:
        return np.zeros(1024, dtype=int)

print("Generator initialized.")

drug_data['fingerprint'] = drug_data['smiles'].apply(get_fingerprint)

fingerprint_matrix = np.stack(drug_data['fingerprint'].values)

fp_df = pd.DataFrame(fingerprint_matrix, index=drug_data.index, dtype='int8')

fp_df.columns = [f'FP_{i}' for i in range(1024)]

drug_data_featurized = pd.concat([drug_data, fp_df], axis=1)

print("Success!")
print(drug_data_featurized.head())

Generator initialized.
Success!
      drug_name  drug_id cell_line_name  cosmic_id tcga_classification  \
0  Camptothecin     1003         PFSK-1     683667                  MB   
1  Camptothecin     1003           A673     684052        UNCLASSIFIED   
2  Camptothecin     1003            ES5     684057        UNCLASSIFIED   
3  Camptothecin     1003            ES7     684059        UNCLASSIFIED   
4  Camptothecin     1003          EW-11     684062        UNCLASSIFIED   

           tissue   tissue_sub_type      ic50       auc  max_conc  ...  \
0  nervous_system   medulloblastoma -1.463887  0.930220       0.1  ...   
1     soft_tissue  rhabdomyosarcoma -4.869455  0.614970       0.1  ...   
2            bone    ewings_sarcoma -3.360586  0.791072       0.1  ...   
3            bone    ewings_sarcoma -5.044940  0.592660       0.1  ...   
4            bone    ewings_sarcoma -3.741991  0.734047       0.1  ...   

   FP_1014  FP_1015 FP_1016 FP_1017 FP_1018  FP_1019  FP_1020  FP_1021  \
0   

## Remove Missing IC50 Rows

In [27]:
# remove missing IC50s and rows where IC50 <=0 
initial_count = len(drug_data_featurized)

drug_data_featurized = drug_data_featurized.dropna(subset=['ic50'])

final_count = len(drug_data_featurized)

print(f"Initial row count: {initial_count}")
print(f"Rows dropped (NAs or <=0): {initial_count - final_count}")
print(f"Final count: {final_count} ({final_count / initial_count * 100:.2f}%)")

Initial row count: 243466
Rows dropped (NAs or <=0): 0
Final count: 243466 (100.00%)


In [28]:
# create target variable using IC50 (molar conversions)
drug_data_featurized['molar_conc_uM'] = np.exp(drug_data_featurized['ic50'])
drug_data_featurized['molar_conc_M'] = drug_data_featurized['molar_conc_uM'] * 1e-6
drug_data_featurized['pic50'] = -np.log10(drug_data_featurized['molar_conc_M'])
drug_data_featurized[['ic50', 'molar_conc_uM', 'molar_conc_M', 'pic50']].head()

,ic50,molar_conc_uM,molar_conc_M,pic50
0,-1.463887,0.231335,2.313352e-07,6.635758
1,-4.869455,0.007678,7.677550e-09,8.114777
2,-3.360586,0.034715,3.471492e-08,7.459484
3,-5.044940,0.006442,6.441844e-09,8.190990
4,-3.741991,0.023707,2.370685e-08,7.625126


## Pivot & Merge

In [29]:
# pivot using cosmic_id and genetic_feature and reset index to remove extra row
genetic_data_pivoted = genetic_data.pivot(
    index="cosmic_id",
    columns="genetic_feature",
    values="is_mutated"
)

# fill missing values as 0 (normal/wild type)
genetic_data_pivoted = genetic_data_pivoted.fillna(0).astype(int)

# reset index
genetic_data_pivoted = genetic_data_pivoted.reset_index()

# optimize data
genetic_data_pivoted[genetic_data_pivoted.columns[1:]] = genetic_data_pivoted[genetic_data_pivoted.columns[1:]].astype('int8')

# show first rows
genetic_data_pivoted.head()

genetic_feature,cosmic_id,ABCB1_mut,ABL2_mut,ACACA_mut,ACVR1B_mut,ACVR2A_mut,ADCY1_mut,AFF4_mut,AHCTF1_mut,AHNAK_mut,...,cnaPANCAN90,cnaPANCAN91,cnaPANCAN92,cnaPANCAN93,cnaPANCAN94,cnaPANCAN95,cnaPANCAN96,cnaPANCAN97,cnaPANCAN98,cnaPANCAN99
0,683667,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,684052,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,684057,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,684059,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,684062,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# merge datasets
combined_data = pd.merge(drug_data_featurized,
                         genetic_data_pivoted,
                         on='cosmic_id',
                         how='inner')

print("Shape of drug_data_featurized:", drug_data_featurized.shape)
print("Shape of genetic_data_pivoted:", genetic_data_pivoted.shape)
print("Shape of combined_data:", combined_data.shape)

Shape of drug_data_featurized: (243466, 1042)
Shape of genetic_data_pivoted: (969, 736)
Shape of combined_data: (243466, 1777)


In [31]:
# get dummies for tissue column
combined_data = pd.get_dummies(combined_data, columns=['tissue'], prefix='tissue')

In [32]:
# replace empty categorical columns with "Unknown"
combined_data['tissue_sub_type'] = combined_data['tissue_sub_type'].fillna('Unknown')
combined_data['tcga_classification'] = combined_data['tcga_classification'].fillna('Unknown')

In [33]:
# drop missing data
initial_count_combined = len(combined_data)

# drop rows with missing values
combined_data = combined_data.dropna()

final_count_combined = len(combined_data)

print(f"Initial merged row count: {initial_count_combined}")
print(f"Rows dropped: {initial_count_combined - final_count_combined}")
print(f"Final count: {final_count_combined} ({final_count_combined / initial_count_combined * 100:.2f}%)")

Initial merged row count: 243466
Rows dropped: 43502
Final count: 199964 (82.13%)


In [34]:
# final NA count
na_summary = pd.DataFrame({
    'Missing Count': combined_data.isna().sum(),
    'Percentage': combined_data.isna().mean().mul(100).round(2)
})

missing_report = na_summary[na_summary['Missing Count'] > 0].sort_values(by='Percentage')
display(missing_report)

,Missing Count,Percentage


In [35]:
# export dataset as .csv for next notebook
combined_data.to_csv("data/merged_data.csv", index=False)